In [1]:
# import packages
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import time

## Length of the code {-}
No restriction

**Delete this section from the report, when using this template.** 

## Data quality check / cleaning / preparation 

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.** An example is given below.

### Data quality check
*By Elton John*

The code below visualizes the distribution of all the variables in the dataset, and their association with the response.

In [7]:
#...Distribution of continuous variables...#

In [8]:
#...Distribution of categorical variables...#

In [9]:
#...Association of the response with the predictors...#

### Data cleaning
*By Xena Valenzuela*

From the data quality check we realized that:

1. Some of the columns that should have contained only numeric values, specifically <>, <>, and <> have special characters such as \*, #, %. We'll remove these characters, and convert the datatype of these columns to numeric.

2. Some of the columns have more than 60% missing values, and it is very difficult to impute their values, as the values seem to be missing at random with negligible association with other predictors. We'll remove such columns from the data.

3. The column `number_of_bedrooms` has some unreasonably high values such as 15. As our data consist of single-family homes in Evanston, we suspect that any value greater than 5 may be incorrect. We'll replace all values that are greater than 5 with an estimate obtained using the $K$-nearest neighbor approach.

4. The columns `house_price` has some unreasonably high values. We'll tag all values greater than 1 billion dollars as "potentially incorrect observation", to see if they distort our prediction / inference later on.

The code below implements the above cleaning.

In [ ]:
#...Code with comments...#

### Data preparation
*By Adam Ruzumna and Judd Moss*

The following data preparation steps helped us to prepare our data for implementing various modeling / validation techniques:

1. Since we need to predict house price, we derived some new predictors *(from existing predictors)* that intuitively seem to be helpuful to predict house price. 

2. We have shuffled the dataset to prepare it for K-fold cross validation.

3. We have created a standardized version of the dataset, as we will use it to develop Lasso / Ridge regression models.

In [3]:
######---------------Creating new predictors----------------#########

#Creating number of bedrooms per unit floor area

#Creating ratio of bathrooms to bedrooms

#Creating ratio of carpet area to floor area

In [ ]:
######-----------Shuffling the dataset for K-fold------------#########

In [ ]:
######-----Standardizing the dataset for Lasso / Ridge-------#########

- Adding in continents to be a categorical variable

In [ ]:
# add in continents to be a categorical variable
continents_df = pd.read_csv('gdp_lifeExpectancy.csv')
continents = continents_df[['country', 'continent']]
continents.head(1)

In [ ]:
data = pd.merge(data, continents, on='country')

In [ ]:
data.head(1)

## Exploratory data analysis

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.**

In [2]:
# Exploring different distributions of life expectancy by continent - Judd Moss
sns.displot(data = train, x = 'life_expectancy',kind = 'kde', hue='continent').set_titles("Life Expectancy Distribution by Continent")
plt.show()

NameError: name 'train' is not defined

In [ ]:
# check the correlation with life_expectancy and the potential predictors - Judd Moss
sns.set(rc={'figure.figsize':(12,10)})
sns.heatmap(train.corr(), cmap='tab10')

- The variable most correlated with life expectancy was `income_composition_of_resources` so we wanted to check its distribution since it was on a 0-1 scale to see where most countries fell on the scale

In [3]:
# check scatterplot of income_composition_of_resources and life expectancy - Judd
ax = sns.scatterplot(data = train, x = 'income_composition_of_resources',y = 'life_expectancy', hue='continent')
ax.set_title("Life Expectancy by Continent")
plt.show()

NameError: name 'train' is not defined

- `income_composition_of_resources` measures on a scale of 0 to 1, a countries ability to utilize its resources. We can see many of the countries with values of `income_composition_of_resources` have a higher life expectancy. 

## Developing the model
*By Judd Moss and Alissa Chu*

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.**

**Addressing Multicollinearity**
- In the following code chunk, we tested the multicollinearity of the predictors that could be used in our regression model. Since many of the healthcare variables were related we suspected the predictors would exhibit some collinearity. To check this, we computed their VIF values

In [ ]:
#Predictors VIF will be caluclated for:

X = train[['adult_mortality','measles', 'under_five_deaths', 'polio', 'diphtheria', 'hiv_aids', 
           'gdp_per_capita', 'thinness_1_19_years', 'thinness_5_9_years',
           'income_composition_of_resources', 'schooling']]

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# add constant column
X = add_constant(X)
# create empty dataframe to input VIF values
vif_data=pd.DataFrame()
# create row for each predictor in X
vif_data['predictor'] = X.columns

# for loop to fill in VIF values 
for i in range(len(X.columns)):
    vif_data.loc[i,'VIF'] = variance_inflation_factor(X.values,i)
    
print(vif_data)

- We then checked which variables were most correlated with `life_expectancy` to decide which variables with high VIF levels to remove. It appears `thinness_1_19_years`, `thinness_5_9_years`, `income_composition_of_resources`, and `schooling` exhibit multicollinearity. Based on their correlations with `life_expectancy`, we will remove `thinness_5_9_years` and `schooling` since they are less correlated with `life_expectancy`. 

**Model Selection**
- We ran a forward stepwise selection algorithm to help determine which variables we should use in our model.

In [ ]:
#Creating a dataframe with all the predictors
X = train[['adult_mortality', 'measles', 'under_five_deaths', 'polio', 'diphtheria', 'hiv_aids', 'gdp_per_capita', 'thinness_1_19_years', 'income_composition_of_resources']]

In [ ]:
#Function to develop a model based on all predictors in predictor_subset
def processSubset(predictor_subset):
    # Fit model on feature_set and calculate R-squared
    model = sm.ols('life_expectancy~' + '+'.join(predictor_subset),data = train).fit()
    Rsquared = model.rsquared
    return {"model":model, "Rsquared":Rsquared}

In [ ]:
#Function to find the best predictor out of p-k predictors and add it to the model containing the k predictors
def forward(predictors):

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]
    
    results = []
    
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p]))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['Rsquared'].argmax()]

    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [ ]:
def forward_selection():
    models_best = pd.DataFrame(columns=["Rsquared", "model"])

    predictors = []

    for i in range(1,len(X.columns)+1):    
        models_best.loc[i] = forward(predictors)
        predictors = list(models_best.loc[i]["model"].params.index[1:])

    return models_best

In [ ]:
models_best = forward_selection()

In [ ]:
# define function to assess which model is the best using AIC, BIC and adjusted R2
def best_sub_plots():
    plt.figure(figsize=(20,10))
    plt.rcParams.update({'font.size': 18, 'lines.markersize': 10})

    # Set up a 2x2 grid so we can look at 4 plots at once
    plt.subplot(2, 2, 1)

    # We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
    # The argmax() function can be used to identify the location of the maximum point of a vector
    plt.plot(models_best["Rsquared"])
    plt.xlabel('# Predictors')
    plt.ylabel('Rsquared')

    # We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
    # The argmax() function can be used to identify the location of the maximum point of a vector

    rsquared_adj = models_best.apply(lambda row: row[1].rsquared_adj, axis=1)

    plt.subplot(2, 2, 2)
    plt.plot(rsquared_adj)
    plt.plot(1+rsquared_adj.argmax(), rsquared_adj.max(), "or")
    plt.xlabel('# Predictors')
    plt.ylabel('adjusted rsquared')

    # We'll do the same for AIC and BIC, this time looking for the models with the SMALLEST statistic
    aic = models_best.apply(lambda row: row[1].aic, axis=1)

    plt.subplot(2, 2, 3)
    plt.plot(aic)
    plt.plot(1+aic.argmin(), aic.min(), "or")
    plt.xlabel('# Predictors')
    plt.ylabel('AIC')

    bic = models_best.apply(lambda row: row[1].bic, axis=1)

    plt.subplot(2, 2, 4)
    plt.plot(bic)
    plt.plot(1+bic.argmin(), bic.min(), "or")
    plt.xlabel('# Predictors')
    plt.ylabel('BIC')
best_sub_plots()

**Result: Based on adjusted R2, AIC, and BIC, it appears the model with all 9 predictors is the best model.**

### Code fitting the final model

Put the code(s) that fit the final model(s) in separate cell(s), i.e., the code with the `.ols()` or `.logit()` functions.

## Conclusions and Recommendations to stakeholder(s)

You may or may not have code to put in this section. Delete this section if it is irrelevant.